In [1]:
from vibegrad import Tensor

In [2]:
a = Tensor(2)
b = Tensor(4)

In [3]:
import numpy as np
c = Tensor(1)
c.data

array(1.)

In [4]:
c = a*b
c.backward()
c.grad

1

In [5]:
b.grad, a.grad

(array(2.), array(4.))

In [6]:
c.zero_grad()
a.zero_grad()
b.zero_grad()

In [7]:
from vibegrad.nn import Linear, Sequential

In [8]:
a = Linear(1, 1)

In [9]:
import numpy as np
np.random.uniform((1,1))

array([1., 1.])

In [10]:
a(Tensor([1]))

Tensor(data=[0.84826593])

In [11]:
a.weight, a.bias

(Tensor(data=[[0.84826593]]), array([0.]))

In [12]:
import torch
from torch.nn import Linear as tLinear

In [13]:
a = tLinear(1,1)

In [14]:
a(torch.tensor([1.0]))

tensor([0.7749], grad_fn=<ViewBackward0>)

In [15]:
a.weight, a.bias

(Parameter containing:
 tensor([[0.1117]], requires_grad=True),
 Parameter containing:
 tensor([0.6632], requires_grad=True))

In [16]:
from vibegrad.nn import *

In [17]:
model = Sequential([
    Linear(256, 300),
    ReLU(),
    Linear(300, 300),
    ReLU(),
    Linear(300, 10),
    Sigmoid()
])
model

In [18]:
model.parameters

'Linear(256, 300, bias=True) <vibegrad.nn.activations.ReLU object at 0x7a9b7c0f8d90>Linear(300, 300, bias=True) <vibegrad.nn.activations.ReLU object at 0x7a9b7c0fbe50>Linear(300, 10, bias=True) <vibegrad.nn.activations.Sigmoid object at 0x7a9b7c0faec0>'

In [19]:
model.total_params()

170410

In [20]:
X = np.random.uniform(-1, 1, size=(100, 256))
y = np.random.choice([0, 1], size=(100, 10), p=[0.5, 0.5])
X.shape
X = Tensor(X)
y = Tensor(y) 
y.data[:5]

array([[0, 0, 1, 0, 1, 1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 1, 1, 1, 0, 1]])

In [21]:
out = model(X)
out.data.shape, y.data.shape

((100, 10), (100, 10))

In [22]:
loss_fn = BCELoss(reduction="mean")
loss = loss_fn(out, y)
print(loss)

Tensor(data=4.98309238881759)


In [23]:
import torch
out = torch.tensor(out.data)
y = torch.tensor(y.data)
loss_fn_torch = torch.nn.BCELoss()
y = y.float()
out = out.float()
loss_fn_torch(out, y)

tensor(13.5513)

In [24]:
a = Tensor([[0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0]])

for ding in a.data:
    for dong in ding:
        if dong != 1 and dong != 0:
            raise ValueError("ding dong")    

In [25]:
type(a.data)

numpy.ndarray

In [33]:
prediction = Tensor(np.array([0.9, 0.4, 0.7, 0.2]))
target = Tensor(np.array([1, 0, 1, 0]))

loss_fn = BCELoss(reduction="sum")
loss = loss_fn(prediction, target)
print(loss)
loss.backward()
prediction.grad

Tensor(data=1.1960046346767592)


array([-1.11111111,  1.66666667, -1.42857143,  1.25      ])

In [27]:
prediction = torch.tensor(np.array([0.9, 0.4, 0.7, 0.2]), requires_grad=True)
target = torch.tensor(np.array([1,0,1,0]), dtype=torch.double, requires_grad=True)
loss_fn_torch = torch.nn.BCELoss(reduction="sum")
loss_torch = loss_fn_torch(prediction, target)
print(loss_torch)
loss_torch.backward()
prediction.grad

tensor(1.1960, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)


tensor([-1.1111,  1.6667, -1.4286,  1.2500], dtype=torch.float64)

In [28]:
model = torch.nn.Sequential()
model.append(torch.nn.Linear(256, 300))
model.append(torch.nn.Linear(300,300))
model.append(torch.nn.Linear(300,10))

Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)

In [29]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_params}")

Total number of parameters: 170410


In [30]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=256, out_features=300, bias=True)
  (1): Linear(in_features=300, out_features=300, bias=True)
  (2): Linear(in_features=300, out_features=10, bias=True)
)>

In [31]:
import vibegrad.nn as nn

In [32]:
nn.relu(Tensor(2.0))

Tensor(data=2.0)